# EDA Assignment – Optimising NYC Taxis
**Name:** M SRUTHI RAJ

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from datetime import datetime

In [ ]:
# Load and sample data
SAMPLE_FRAC = 0.05
months = [f"{m:02}" for m in range(1, 13)]
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-{}.parquet"
final_sampled_df = pd.DataFrame()

for month in tqdm(months, desc="Processing Monthly Files"):
    try:
        url = base_url.format(month)
        df = pd.read_parquet(url)

        df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
        df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
        df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

        sampled_data = pd.DataFrame()
        for date in df['pickup_date'].unique():
            day_data = df[df['pickup_date'] == date]
            for hour in range(24):
                hour_data = day_data[day_data['pickup_hour'] == hour]
                if not hour_data.empty:
                    sample = hour_data.sample(frac=SAMPLE_FRAC, random_state=42)
                    sampled_data = pd.concat([sampled_data, sample])

        sampled_data.drop(columns=['pickup_date', 'pickup_hour'], inplace=True)
        final_sampled_df = pd.concat([final_sampled_df, sampled_data], ignore_index=True)
    except Exception as e:
        print(f"Failed to process month {month}: {e}")

df = final_sampled_df.copy()
df.reset_index(drop=True, inplace=True)

In [ ]:
# Data cleaning
if 'airport_fee.1' in df.columns:
    df['airport_fee'] = df['airport_fee'].fillna(0) + df['airport_fee.1'].fillna(0)
    df.drop(columns=['airport_fee.1'], inplace=True)

df['passenger_count'].replace(0, np.nan, inplace=True)
df['passenger_count'].fillna(1, inplace=True)
df['RatecodeID'].fillna(1, inplace=True)
df['congestion_surcharge'].fillna(0, inplace=True)
df = df[df['payment_type'].between(1, 6)]
df = df[df['trip_distance'].between(0.1, 250)]
df = df[df['fare_amount'] >= 0]
df = df[df['total_amount'] >= 0]

In [ ]:
# Feature engineering
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['pickup_month'] = df['tpep_pickup_datetime'].dt.month

In [ ]:
# Visualization: Hourly pickups
hourly_counts = df['pickup_hour'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
sns.lineplot(x=hourly_counts.index, y=hourly_counts.values, marker='o')
plt.title('Taxi Pickups by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Pickups')
plt.grid(True)
plt.savefig('chart_hourly_pickups.png')
plt.show()